In [1]:
%%bash
PROJECT_ID=$(gcloud config get-value project)
DATASET="challenge3"

bq --location=US mk -d \
    --description "emergency calls response times" \
    "${PROJECT_ID}:${DATASET}" || true

Dataset 'qwiklabs-gcp-00-46c4d2064c57:challenge3' successfully created.


In [2]:
import os
PROJECT_ID = "qwiklabs-gcp-00-46c4d2064c57"
DATASET = "challenge3"
os.environ['PROJECT_ID'] = PROJECT_ID
os.environ['DATASET'] = DATASET

In [3]:
!bq load \
  --source_format=CSV \
  --skip_leading_rows=1 \
  --autodetect \
  $PROJECT_ID:$DATASET.weather_data_raw \
  gs://labs.roitraining.com/data-to-ai-workshop/weather_data.csv

Waiting on bqjob_r248dee365219e3fb_0000019bc2fd951f_1 ... (1s) Current status: DONE   


In [4]:
#gemini-external-model
import subprocess, json, re

LOCATION = "US"
CONNECTION_ID = "gemini_conn"

# Create the connection
subprocess.run([
    "bq", f"--project_id={PROJECT_ID}", "mk", "--connection",
    f"--location={LOCATION}",
    "--connection_type=CLOUD_RESOURCE",
    CONNECTION_ID
], check=False)

# Show the connection and extract the service account
out = subprocess.check_output([
    "bq", f"--project_id={PROJECT_ID}", "show", "--format=prettyjson",
    "--connection", f"{PROJECT_ID}.{LOCATION}.{CONNECTION_ID}"
], text=True)

conn = json.loads(out)
sa = conn["cloudResource"]["serviceAccountId"]
print("Connection service account:", sa)

Connection service account: bqcx-398662785422-l0et@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [5]:
# Grant Vertex AI User role to the connection service account
subprocess.check_call([
    "gcloud", "projects", "add-iam-policy-binding", PROJECT_ID,
    "--member", f"serviceAccount:{sa}",
    "--role", "roles/aiplatform.user"
])

print("Granted roles/aiplatform.user to:", sa)

Granted roles/aiplatform.user to: bqcx-398662785422-l0et@gcp-sa-bigquery-condel.iam.gserviceaccount.com


In [7]:
#gemini model
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

DATASET = "challenge3"
MODEL_NAME = f"{PROJECT_ID}.{DATASET}.gemini_model"
CONN_REF = f"{LOCATION}.{CONNECTION_ID}"

create_model_sql = f"""
CREATE OR REPLACE MODEL `{MODEL_NAME}`
REMOTE WITH CONNECTION `{CONN_REF}`
OPTIONS (
  ENDPOINT = 'gemini-2.5-flash'
)
"""
client.query(create_model_sql).result()
print("Created remote model:", MODEL_NAME)

Created remote model: qwiklabs-gcp-00-46c4d2064c57.challenge3.gemini_model


In [18]:
#generate report
WEATHER_TABLE = f"{PROJECT_ID}.{DATASET}.weather_data_raw"
OUT_TABLE = f"{PROJECT_ID}.{DATASET}.weather_data_with_report" # create a new table to store
MODEL_NAME = f"{PROJECT_ID}.{DATASET}.gemini_model"

sql = f"""
CREATE OR REPLACE TABLE `{OUT_TABLE}` AS
SELECT
  JSON_VALUE(ml_generate_text_result, '$.candidates[0].content.parts[0].text') AS weather_report,
  * EXCEPT(prompt, ml_generate_text_result, ml_generate_text_status)
FROM ML.GENERATE_TEXT(
  MODEL `{MODEL_NAME}`,
  (
    SELECT
      CONCAT(
        'Create a short weather report (2-4 sentences) and any warnings if needed based on this record: ',
        TO_JSON_STRING(t)
      ) AS prompt,
      t.*
    FROM `{WEATHER_TABLE}` AS t
  ),
  STRUCT(0.2 AS temperature, 2000 AS max_output_tokens)
);
"""
client.query(sql).result()
print("Report created, output is written to:", OUT_TABLE)


Report created, output is written to: qwiklabs-gcp-00-46c4d2064c57.challenge3.weather_data_with_report


In [19]:
#display table output for grading
q1 = f"""
SELECT *
FROM `{PROJECT_ID}.{DATASET}.weather_data_with_report`
limit 20
"""
df_display = client.query(q1).to_dataframe()
df_display

,weather_report,date,city,state,temperature_f,wind_speed_mph,precipitation_in,barometric_pressure_inHg,humidity_percent,weather_condition
0,"Good morning, Atlanta! On March 10, 2025, expe...",2025-03-10,Atlanta,GA,59.5,9.6,0.09,29.67,57.2,Cloudy
1,"Good morning, Atlanta! On March 14, 2025, expe...",2025-03-14,Atlanta,GA,71.7,7.2,0.18,29.92,55.3,Cloudy
2,"Good morning, Atlanta! On March 1st, expect cl...",2025-03-01,Atlanta,GA,51.7,4.7,0.08,29.74,49.9,Cloudy
3,"Good morning, Atlanta! On March 5, 2025, expec...",2025-03-05,Atlanta,GA,74.4,5.1,0.02,29.92,50.4,Cloudy
4,"Atlanta, GA is experiencing a warm and cloudy ...",2025-02-26,Atlanta,GA,75.2,10.4,0.03,29.58,49.9,Cloudy
5,"Good morning, Atlanta! On February 21, 2025, e...",2025-02-21,Atlanta,GA,55.7,5.0,0.12,29.80,50.4,Cloudy
6,"Good morning, Boston! On February 19, 2025, ex...",2025-02-19,Boston,MA,61.7,3.9,0.11,29.62,54.1,Cloudy
7,"Good morning, Boston! On Thursday, March 13, 2...",2025-03-13,Boston,MA,71.9,9.8,0.16,29.99,42.3,Cloudy
8,"Good morning, Boston! Today, March 19, 2025, e...",2025-03-19,Boston,MA,60.7,6.4,0.04,29.83,49.4,Cloudy
9,"Good morning, Boston! On March 9, 2025, expect...",2025-03-09,Boston,MA,76.7,4.3,0.09,29.52,40.9,Cloudy
